# Data Organization

Organizing the data, and splitting into train and test set. 

TODO: 
* probably want to write some functions in python so we can call these wherever

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
raw_data_dir = '../data/raw/'
meta_data_dir = '../data/meta/'

In [3]:
!ls {raw_data_dir}

dim_claims.csv dim_date.csv   dim_pa.csv


In [4]:
!ls {meta_data_dir}

DataDictionary.txt bridge.csv


# Raw Data

## Claims Data

This is a table containing pharmacy claim-level information.

dim_claims.csv

* It contains the following columns:

* dim_claim_id: Primary key and index. Integer.
* bin: The BIN of the payer for the claim. Numeric.
* drug: The drug that was associated with the claim. Character.
* reject_code: If the claim was rejected, what was the associated rejection code. Character.
* pharmacy_claim_approved: Flag for if the claim was approved
by the payer. Bit.

In [5]:
claims_filename = 'dim_claims.csv'
claims = pd.read_csv(f'{raw_data_dir}{claims_filename}')

In [6]:
claims.head()

,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
0,1,417380,A,75.0,0
1,2,999001,A,NaN,1
2,3,417740,A,76.0,0
3,4,999001,A,NaN,1
4,5,417740,A,NaN,1


Phama

In [7]:
len(claims[claims.pharmacy_claim_approved.values==0])

555951

Check for NaNs.

In [8]:
claims.isna().any()

dim_claim_id               False
bin                        False
drug                       False
reject_code                 True
pharmacy_claim_approved    False
dtype: bool

Looks like the NaNs are from reject codes where the pharmacy claim was approved.

In [9]:
claims[claims.pharmacy_claim_approved.values==0].isna().any()

dim_claim_id               False
bin                        False
drug                       False
reject_code                False
pharmacy_claim_approved    False
dtype: bool

In [10]:
claims.notnull().sum()

dim_claim_id               1335576
bin                        1335576
drug                       1335576
reject_code                 555951
pharmacy_claim_approved    1335576
dtype: int64

In [11]:
claims['reject_code'] = claims.reject_code.fillna(0).astype(int)

In [12]:
# bin is an identifier for the payer, so we don't want to misconstrue the numbers to have numerical meaning
claims['bin'] = claims.bin.astype(str)

In [13]:
type(claims['bin'][0])

str

## PA Data

This is a table containing PA-level for every rejected pharmacy claim in
dim_claims.

dim_pa.csv


* It contains the following columns:

* dim_pa_id: Primary key and index. Integer.
* correct_diagnosis: Flag for information provided by the provider
indicating that the patient has the correct diagnosis for the
* associated drug. Bit.
* tried_and_failed: Flag for information provided by the provider
indicating that the patient has tried and failed the relevant
* generic alternatives. Bit.
* contraindication: Flag for information provided by the provider
indicating that the patient has an associated contraindication for
the medication requested. Bit.
* pa_approved: Flag for if the payer favorably reviewed and
approved the PA. Bit.

In [14]:
pa_filename = 'dim_pa.csv'
pa = pd.read_csv(f'{raw_data_dir}{pa_filename}')

In [15]:
pa.head(5)

,dim_pa_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved
0,1,1,1,0,1
1,2,1,0,0,1
2,3,0,0,1,1
3,4,1,1,0,1
4,5,0,1,0,1


In [16]:
pa.isna().any()

dim_pa_id            False
correct_diagnosis    False
tried_and_failed     False
contraindication     False
pa_approved          False
dtype: bool

In [17]:
pa.notnull().sum()

dim_pa_id            555951
correct_diagnosis    555951
tried_and_failed     555951
contraindication     555951
pa_approved          555951
dtype: int64

## Date Data

This is a table containing date-level information that might be useful for
producing transaction forecasts.

dim_date.csv

* It contains the following columns:

* dim_date_id: Primary key and index. Integer.
* date_val: Date value corresponding to the record. Date.
* calendar_year: Year for the record. Integer.
* calendar_month: Month of the year for the record. Integer.
* calendar_day: Day of the month for the record. Integer.
* day_of_week: The Nth day of the week. Integer.
* is_weekday: Flag for if the record is associated with Monday
through Friday. Bit.
* is_workday: Flag for if the record is associated with Monday
through Friday and is not a holiday. Bit.
* is_holiday: Flag for if the record is associated with a holiday.
Bit.

In [18]:
date_filename = 'dim_date.csv'
dates = pd.read_csv(f'{raw_data_dir}{date_filename}')

In [19]:
dates.head()

,dim_date_id,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday
0,1,2017-01-01,2017,1,1,1,0,0,1
1,2,2017-01-02,2017,1,2,2,1,0,0
2,3,2017-01-03,2017,1,3,3,1,1,0
3,4,2017-01-04,2017,1,4,4,1,1,0
4,5,2017-01-05,2017,1,5,5,1,1,0


In [20]:
dates.isna().any()

dim_date_id       False
date_val          False
calendar_year     False
calendar_month    False
calendar_day      False
day_of_week       False
is_weekday        False
is_workday        False
is_holiday        False
dtype: bool

In [21]:
dates.notnull().sum()

dim_date_id       1520
date_val          1520
calendar_year     1520
calendar_month    1520
calendar_day      1520
day_of_week       1520
is_weekday        1520
is_workday        1520
is_holiday        1520
dtype: int64

# Meta Data

## Bridge Data

bridge.csv
* This is a bridge table that links the primary keys of all the tables to one
another that is used for joining the tables.
* It contains the following columns:

* dim_claim_id: Primary key for dim_claims.
* dim_pa_id: Primary key for dim_pa.
* dim_date_id: Primary key for dim_date.


Note: a PA could have more than one claim, a date could also have more than one claim, the largest dataset are the claims.

In [22]:
bridge_filename = 'bridge.csv'
bridge = pd.read_csv(f'{meta_data_dir}{bridge_filename}')

In [23]:
def test_join(df):
    """Do a test that we did the bridge correctly."""
    
    # check that we have the right shape
    
    shape = df.shape
    
    if shape==(1335576, 19):
        print(f'Dataframe has the correct shape of {shape}')
    else:
        print(f'Oops! Dataframe has a shape of {shape}, expected {(1335576, 3)}.')
        
    # check that we have the expected number of NaN's (only the number of ones in the bridge file).
    
    num_nans = len(df) - len(df.dropna())
    exp_num_nans = len(bridge)-len(bridge.dropna())
    
    if num_nans==exp_num_nans:
        print("Correct amount of NaN's in the dataframe.")
    else:
        print(f"Oops! Datamframe has {num_nans} NaN's and {exp_num_nans} Nans in the bridge file.")
    
    return 

In [24]:
bridge.isna().any()

dim_claim_id    False
dim_pa_id        True
dim_date_id     False
dtype: bool

In [25]:
bridge.notnull().sum()

dim_claim_id    1335576
dim_pa_id        555951
dim_date_id     1335576
dtype: int64

How many unique claim ID's do we have? 

In [26]:
len(np.unique(bridge['dim_claim_id']))

1335576

The are some NaN's in the PA data, does leave the same amount of PA's that we have in the CSV file? 

In [27]:
cut = ~np.isnan(bridge['dim_pa_id'])
len(bridge['dim_pa_id'][cut])

555951

In [28]:
pa.shape

(555951, 5)

In [29]:
dates.shape

(1520, 9)

In [30]:
claims.shape

(1335576, 5)

In [31]:
claims.head()

,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
0,1,417380,A,75,0
1,2,999001,A,0,1
2,3,417740,A,76,0
3,4,999001,A,0,1
4,5,417740,A,0,1


Result will be a csv of claims, with length of 1335576, some PA data will be missing we can leave these as NaN's for now. If we include all the columns that we have some data for (inlcuding the key as a column) we should have 17 columns.

In [32]:
data = pd.DataFrame()

In [33]:
test_join(bridge)

Oops! Dataframe has a shape of (1335576, 3), expected (1335576, 3).
Correct amount of NaN's in the dataframe.


First we want to match the dates data with the claims data, note more than one date might correspond to one claim.

In [34]:
# adding the bridge id's to each data frame

claims = claims.merge(bridge, on='dim_claim_id')
dates = dates.merge(bridge, on='dim_date_id')
pa = pa.merge(bridge, on='dim_pa_id')

In [35]:
dates = dates.merge(claims, on='dim_claim_id', how='outer')
data = dates.merge(pa, on='dim_claim_id', how='outer')

In [36]:
# Check the matching is correct

def check_matching(df):
    
    for col in df.columns:
        if '_x' in col:
            if np.nansum(df[col].values-df[col[:-2]+'_y'].values)==0:
                # delete the repeat columns 
                del df[col]
                del df[col[:-2]+'_y']
            else:
                print(f'Oops! Matching in column, {col} not correct.')
                return
    print('correctly matched')

In [37]:
check_matching(data)

correctly matched


In [38]:
data

,date_val,calendar_year,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved,dim_pa_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved,dim_date_id
0,2017-01-01,2017,1,1,1,0,0,1,1,417380,A,75,0,1.0,1.0,1.0,0.0,1.0,1.0
1,2017-01-01,2017,1,1,1,0,0,1,2,999001,A,0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01,2017,1,1,1,0,0,1,3,417740,A,76,0,2.0,1.0,0.0,0.0,1.0,1.0
3,2017-01-01,2017,1,1,1,0,0,1,4,999001,A,0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01,2017,1,1,1,0,0,1,5,417740,A,0,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335571,2019-12-31,2019,12,31,3,1,1,0,1335572,417740,C,75,0,555950.0,1.0,0.0,0.0,1.0,1095.0
1335572,2019-12-31,2019,12,31,3,1,1,0,1335573,999001,C,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1335573,2019-12-31,2019,12,31,3,1,1,0,1335574,417380,C,70,0,555951.0,0.0,0.0,1.0,0.0,1095.0
1335574,2019-12-31,2019,12,31,3,1,1,0,1335575,999001,C,0,1,NaN,NaN,NaN,NaN,NaN,NaN


## Test Train Split 

How would we like to split the data? Chronologically? To include a similar number of reject code types? Or other categorical types? 

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
data.shape

(1335576, 19)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(data, data['pharmacy_claim_approved'], test_size=0.33, random_state=42)

In [42]:
X_train.to_csv('../data/training/train.csv')
X_test.to_csv('../data/training/test.csv')